In [ ]:
!python --version
!nvidia-smi
!sudo apt-get -y install espeak-ng
!sudo apt-get install python3.9-distutils
!sudo apt-get install python3.9
!pip -q install uv
!uv pip -q install virtualenv --system


In [ ]:
!virtualenv -p python3.9 my_env
!source my_env/bin/activate; pip list
!source my_env/bin/activate; pip -q install uv
!source my_env/bin/activate; uv pip -q install coqui-tts==0.25.1 

In [ ]:
!mkdir train_output
!mkdir train_output/kagg

In [ ]:
!wget https://raw.githubusercontent.com/coqui-ai/TTS/main/TTS/bin/collect_env_info.py
!source my_env/bin/activate; python collect_env_info.py

در کد زیر مسیر دیتاست و 
sample_rate فایل های صوتی تان را ویرایش کنید


In [ ]:
code='''import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig , CharactersConfig
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.utils.downloaders import download_thorsten_de

output_path = os.path.dirname(os.path.abspath(__file__))
dataset_config = BaseDatasetConfig(
    formatter="mozilla", meta_file_train="metadata.csv", path="Path to your dataset" 
)



audio_config = BaseAudioConfig(
    sample_rate=22050,
    do_trim_silence=False,
    resample=False,
    mel_fmin=0,
    mel_fmax=None 
)
character_config=CharactersConfig(
  characters='ءابتثجحخدذرزسشصضطظعغفقلمنهويِپچژکگیآأؤإئًَُّ',
  punctuations='!(),-.:;? ̠،؛؟‌<>',
  phonemes='ˈˌːˑpbtdʈɖcɟkɡqɢʔɴŋɲɳnɱmʙrʀⱱɾɽɸβfvθðszʃʒʂʐçʝxɣχʁħʕhɦɬɮʋɹɻjɰlɭʎʟaegiouwyɪʊ̩æɑɔəɚɛɝɨ̃ʉʌʍ0123456789"#$%*+/=ABCDEFGHIJKLMNOPRSTUVWXYZ[]^_{}',
  pad="<PAD>",
  eos="<EOS>",
  bos="<BOS>",
  blank="<BLNK>",
  characters_class="TTS.tts.utils.text.characters.IPAPhonemes",
  )
config = VitsConfig(
    audio=audio_config,
    run_name="vits_fa_male",
    batch_size=16,
    eval_batch_size=8,
    batch_group_size=5,
    num_loader_workers=0,
    num_eval_loader_workers=2,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    save_step=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=True,
    phoneme_language="fa",
    characters=character_config,
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=250,
    print_eval=False,
    mixed_precision=False,
    test_sentences=[
        ["سلطان محمود در زمستانی سخت به طلخک گفت "],
        ["کارل و لرل کارها رو رله کردن "],
        ["مردی نزد بقالی آمد و گفت پیاز هم ده تا دهان بدان خو شبوی سازم."],
        ["سه سیر سرشیر سه شیشه شیر! "],
        ["از مال خود پاره ای گوشت بستان و زیره بایی معطّر بساز"],
        ["لورل روی ریل راه میرفت "],
        ["یکی اسبی به عاریت خواست"]
    ],
    output_path=output_path,
    datasets=[dataset_config],
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()'''
f=open("train_output/train_vits.py","w",encoding="utf-8")

f.write(code)

f.close()

مدلی که می خواهید روی آن finetune 
کنید اینجا قرار دهید

In [ ]:
!wget "https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/checkpoint_166000.pth?download=true" \
    -O train_output/kagg/checkpoint_166000.pth


In [ ]:
!ls -ul 'train_output'
!ls -ul /kaggle/working/

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!source my_env/bin/activate; PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:512" python "train_output/train_vits.py" \
--restore_path "train_output/kagg/checkpoint_166000.pth" \
--coqpit.run_name "vits-male-finetune" 